In [10]:
!apt-get update
!apt-get install -y poppler-utils

Get:1 https://cli.github.com/packages stable InRelease [3,917 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 https://cli.github.com/packages stable/main amd64 Packages [346 B]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,006 kB]
Hit:8 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,795 kB]
Hit:12 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:13 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu

# Task
Create a Python script that loads a PDF, splits it into chunks, creates a Pinecone vector store from the chunks using a custom embedding model, sets up a multi-query retriever and a conversational retrieval chain with memory, and includes a function to clean the output.

## Install necessary libraries

### Subtask:
Install all the required Python libraries.


**Reasoning**:
The first step is to install the required libraries using pip.



In [23]:
!pip install --q langchain_openai
!pip install --q langchain
!pip install --q unstructured langchain
!pip install --q "unstructured[all-docs]"
!pip install --q langchain_community
!pip install --q chromadb
!pip install --q langchain_text_splitters
!pip install --q poppler-utils
!pip install --q pinecone
!pip install --q langchain_pinecone

## Set up environment variables

### Subtask:
Set up the API keys for OpenAI and Pinecone.


**Reasoning**:
Set the environment variables for OpenAI and Pinecone API keys and the OpenAI API base URL.



In [24]:
import os

os.environ['OPENAI_API_KEY'] = "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"
os.environ['OPENAI_API_BASE'] = "https://openrouter.ai/api/v1"
os.environ['PINECONE_API_KEY'] = "pcsk_74vokm_KUuxFinVGUYHVxjtpyxwMixzQVV9whb7TCyUpUs1JjPkxupr95GShq4Fx8e6USz"

## Define embedding model

### Subtask:
Define the custom embedding model using SentenceTransformer.


**Reasoning**:
Define the custom embedding model class and instantiate it as per the instructions.



In [25]:
from langchain_core.embeddings import Embeddings
from sentence_transformers import SentenceTransformer
from typing import List

class SentenceTransformerEmbeddingsWrapper(Embeddings):
    def __init__(self, model_name: str):
        self.model = SentenceTransformer(model_name)

    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        return self.model.encode(texts).tolist()

    def embed_query(self, text: str) -> List[float]:
        return self.model.encode(text).tolist()

model_name = "latterworks/ollama-embeddings"
embeddings_model = SentenceTransformerEmbeddingsWrapper(model_name)

## Load and split pdf

### Subtask:
Load the PDF document and split it into smaller chunks.


**Reasoning**:
Load the PDF and split it into chunks according to the instructions.



In [26]:
from langchain_community.document_loaders import UnstructuredPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=8000, chunk_overlap=100)
pdf_paths = ["/content/Persuader PDF.pdf"]
all_chunks = []

for pdf_path in pdf_paths:
    print(f"Processing {pdf_path}...")
    loader = UnstructuredPDFLoader(pdf_path)
    data = loader.load()
    chunks = splitter.split_documents(data)
    for chunk in chunks:
        chunk.metadata["source_file"] = pdf_path
    all_chunks.extend(chunks)

print(f"Processed {len(pdf_paths)} PDF(s) and created {len(all_chunks)} chunks.")

Processing /content/Persuader PDF.pdf...
Processed 1 PDF(s) and created 17 chunks.


## Initialize pinecone

### Subtask:
Initialize the Pinecone client and define the index name.


**Reasoning**:
Initialize the Pinecone client and define the index name as requested by the subtask.



In [27]:
from pinecone import Pinecone

# 1. Initialize Pinecone client
api_key = os.environ.get('PINECONE_API_KEY')
pc = Pinecone(api_key=api_key)

# 2. Define the index name
index_name = "apirag"

print(f"Pinecone client initialized. Index name set to '{index_name}'.")

Pinecone client initialized. Index name set to 'apirag'.


## Create pinecone vector store

### Subtask:
Create a Pinecone vector store from the document chunks and embeddings.


**Reasoning**:
Create a Pinecone vector store from the document chunks and embeddings using the PineconeVectorStore.from_documents method.



In [28]:
from langchain_pinecone import PineconeVectorStore

vector_store = PineconeVectorStore.from_documents(
    all_chunks,
    index_name=index_name,
    embedding=embeddings_model
)

print("Pinecone vector store created successfully.")

Pinecone vector store created successfully.


## Set up multiqueryretriever

### Subtask:
Set up a retriever that generates multiple queries from the user's question.


**Reasoning**:
Define the prompt template for generating multiple queries and set up the MultiQueryRetriever using the vector store and LLM.



In [29]:
from langchain.prompts import PromptTemplate
from langchain.retrievers.multi_query import MultiQueryRetriever

query_prompt = PromptTemplate(
    input_variables=["question"],
    template="""You are an AI language model assistant. Your task is to generate
     five different versions of the given user question to retrieve relevant documents
    from a vector database. By generating multiple perspectives on the user question,
    your goal is to help the user overcome some of the limitations of the distance
    based similarity search. Provide these alternative questions separated by new lines.
    Original question: {question}""",
)

retriever = MultiQueryRetriever.from_llm(
    vector_store.as_retriever(),
    llm,
    prompt=query_prompt,
)

## Create conversationalretrievalchain

### Subtask:
Create a RAG chain that uses the retriever and an LLM to answer questions with memory.


**Reasoning**:
Import the necessary classes for the conversational retrieval chain and define the RAG prompt template.



In [57]:
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

#Rag prompt
template = """Answer the question based only on the following context and also give source snippets from the pdf below the answer:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

**Reasoning**:
Initialize the conversation memory and create the conversational retrieval chain with memory, then define the separate chain without memory for direct querying.



In [58]:
# Initialize ConversationBufferMemory
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# Create a conversational retrieval chain
chain_with_memory = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    memory=memory,
    combine_docs_chain_kwargs={"prompt": prompt}
)

# The original chain without memory is still defined here for reference
chain = (
    {"context" : retriever, "question" : RunnablePassthrough()}
    | prompt | llm | StrOutputParser()
)

## Define output cleaning function

### Subtask:
Define a function to clean the output string.


**Reasoning**:
Define the clean_output function as instructed to format the output string.



In [59]:
def clean_output(output_string):
    # Replace '\n\n' with actual newlines and format list items
    formatted_output = output_string.replace('\n\n', '\n')
    lines = formatted_output.split('\n')
    processed_lines = []
    for line in lines:
        # Remove '*' and leading/trailing spaces
        processed_line = line.lstrip('* ').strip()
        processed_lines.append(processed_line)

    final_output = '\n'.join(processed_lines)
    return final_output

# Task
Integrate a Pinecone reranker into the existing LangChain conversational retrieval chain and the runnable chain, ensuring the reranking step occurs after document retrieval and before answer generation.

## Initialize pinecone reranker

### Subtask:
Initialize the Pinecone reranker model.


**Reasoning**:
Initialize the Pinecone reranker model using the Pinecone API key and specify the number of top results.



In [60]:
from langchain_pinecone import PineconeRerank

# Assuming PINECONE_API_KEY is already set as an environment variable
reranker = PineconeRerank(api_key=os.environ.get('PINECONE_API_KEY'), top_n=5)

print("Pinecone reranker initialized.")

Pinecone reranker initialized.


## Modify the rag chain

### Subtask:
Update the existing conversational retrieval chain to include the reranking step after document retrieval and before answer generation.


**Reasoning**:
Update the existing conversational retrieval chain to include the reranking step after document retrieval and before answer generation.



In [61]:
from langchain_core.runnables import RunnableLambda

# Define a custom runnable that retrieves documents and then reranks them
def retrieve_and_rerank(input_dict):
    question = input_dict["question"]
    chat_history = input_dict["chat_history"]

    # Get relevant documents using the base retriever
    retrieved_docs = retriever.get_relevant_documents(question)

    # Rerank the retrieved documents
    reranked_docs = reranker.rerank(retrieved_docs, question)

    return {"question": question, "context": reranked_docs, "chat_history": chat_history}

# Create a new conversational retrieval chain with the custom runnable
chain_with_memory_reranked = (
    RunnableLambda(retrieve_and_rerank)
    | prompt
    | llm
    | StrOutputParser()
)

print("Conversational retrieval chain with reranking created successfully using a custom runnable.")

Conversational retrieval chain with reranking created successfully using a custom runnable.


## Update the runnable chain

### Subtask:
Update the runnable chain to include the reranking step.


**Reasoning**:
Modify the existing runnable chain to include the reranking step by inserting the reranker after the retriever and before the prompt.



**Reasoning**:
The `PineconeRerank` object is not a standard Runnable and cannot be directly chained using the `|` operator. A custom runnable or a LangChain Expression Language (LCEL) chain is needed to integrate the reranking step. Since the goal is to modify the *existing* runnable chain structure, a `RunnableLambda` can be used to wrap the retrieval and reranking logic, similar to how it was done for the conversational chain.



In [62]:
from langchain_core.runnables import RunnableLambda

# Update the original chain without memory to include the reranker
# Use a RunnableLambda to wrap the retriever and reranker
chain = (
    {"context": RunnableLambda(lambda x: reranker.rerank(retriever.invoke(x['question']), x['question'])),
     "question": RunnablePassthrough()}
    | prompt | llm | StrOutputParser()
)

print("Runnable chain updated successfully to include the reranker using RunnableLambda.")

Runnable chain updated successfully to include the reranker using RunnableLambda.


In [63]:
# Get user input
user_question = input("Enter your question about the document: ")

# Invoke the conversational retrieval chain with the user's question
# Assuming the reranked chain is named 'chain_with_memory_reranked' or similar after the previous steps
# If you want to use the original chain without memory but with reranking, use 'chain'
# For this example, let's use the conversational chain with reranking
result = chain_with_memory_reranked.invoke({"question": user_question, "chat_history": memory.load_memory_variables({})['chat_history']})


# Clean the output
cleaned_answer = clean_output(result)

# Print the cleaned answer
print("\nAnswer:")
print(cleaned_answer)

Enter your question about the document: which character is kind of an antihero in the novel

Answer:
Answer**
The character who functions as the novel’s anti‑hero is **Jack Reacher**.
Reacher is a former military policeman who operates largely outside the law, yet he is driven by a personal code of honor and a strong sense of empathy toward those he protects. This blend of violent capability and moral complexity is what makes him an anti‑hero.
Source snippets**
- “The protagonist’s emotional reaction to the aftermath of violence signifies a deep sense of empathy and respect for human life, even amidst his violent world.”
(Doc 0, Q&A section)
- “The protagonist’s willingness to face personal danger in order to help his colleagues and seek justice for the fallen reflects a profound commitment to personal sacrifice.”
(Doc 0, Q&A section)
These passages highlight Reacher’s moral ambiguity and his willingness to act outside conventional law‑making, hallmarks of an anti‑hero.
